In [1]:

from bs4 import BeautifulSoup                        # Importamos librerías para extracción y manipulación de datos 
import requests
import pandas as pd
import re

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.
pd.set_option('display.width', 1000)        # Ajusta el ancho de la consola para evitar truncamientos


# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

In [3]:

def llamar_api (year, genero):                       
    
    # Este método intenta llamar a la API Cinema Movies. 
    # Recibe como parámetro el género y el año en que desea buscar y devuelve una lista de tuplas con los datos de cada película.

    #Construimos la URL dinámica con los parámetros recibidos:

    url = f"https://dev.adalab.es/api/cinema/movies?year={year}&genre={genero}" 

    lista_peliculas = []

    respuesta = requests.get(url)

    sleep(3)

    if respuesta.status_code != 200:
        print(f"Error al haber la llamada. Motivo: {respuesta.reason}")
        
    else: 
        json_genero_year= respuesta.json()

        # Recorro el diccionario que me devuelve la API hasta llegar al diccionario que almacena 
        # los títulos, ids, tipos y años de estreno. Lo itero y extraigo los datos de cada película. 
            
        for pelicula in json_genero_year["results"]:   
                                       
            type = pelicula["type"]

            if type == "Movie":     # Filtramos por película. Ignoramos series y otros formatos de vídeo.

                title = pelicula["title"]
                year = pelicula["year"]
                genero = pelicula ["genre"]
                id = pelicula["id"]   
                idOwner = pelicula["idOwner"]

                datos_pelicula = (type, title, year, genero, id, idOwner)           # En cada iteración, creo una tupla con los datos que añado a mi lista. 
                lista_peliculas.append(datos_pelicula)
            
            else:
                pass        

        return lista_peliculas 
    


In [4]:

def todas_las_peliculas_del_anyo (year) :

    # Devuelve las películas de todos los géneros del año pasado por parámetro.
    # Devuelve una lista de tuplas con los datos de cada película.
    # Además, nos será útil para obtener los identificadores de las películas.

    generos = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]
    lista_pelis_total= []

    for genero in generos: 
        lista_pelis_genero = llamar_api(year, genero) 
        lista_pelis_total.extend(lista_pelis_genero)

    return lista_pelis_total

In [ ]:

def extractor_idowners (archivo_csv) :

    # Esta función recibe un archivo .csv y devuelve una lista con todos los idowners que hay en el archivo.

    df_pelis_API = pd.read_csv(archivo_csv) 
    lista_idowners = df_pelis_API['IdOwner'].tolist()
    return lista_idowners


In [7]:
def extraer_identificadores (lista):
    
    # Devuelve una lista con los identificadores de las películas que recibe por parámetro. 
    # Nos será útil para las funciones con Selenium.

    lista_identificadores = []

    for tupla in lista:
        lista_identificadores.append(tupla[-1]) # En la última posición almacenamos el identificador idOwner.

    return lista_identificadores

In [8]:
def escribir_fichero (df, year_inicial, periodo):

    # Recibe un dataframe, lo pasa a .csv y personaliza el nombre del fichero en función del periodo abarcado.

    if periodo == 1:
        nombre_archivo = f'peliculas_{year_inicial}.csv'
        # df.to_csv(f'peliculas_{year_inicial}.csv') 
        df.to_csv(nombre_archivo) 

    else:
        nombre_archivo = f'peliculas_{year_inicial}_{year_inicial+periodo-1}.csv'
        # df.to_csv(f'peliculas_{year_inicial}_{year_inicial+periodo-1}.csv')   
        df.to_csv(nombre_archivo)           

    print(f"Extracción completa. Puedes consultar tu archivo {nombre_archivo}")

In [9]:

def extraer_datos_peliculas_periodo (year_inicial, periodo = 1):   
    
    # Devuelve un dataframe con los datos de todas las películas del periodo pasado por parámetro.
    # Escribe los datos del dataframe en un fichero. csv con título personalizado.
    # Se recomienda que periodo no exceda de 5 años.

    lista_periodo = []

    if (periodo > 0) and (year_inicial >= 1900):                  

        # Iteramos en los años del periodo marcado y extraemos los datos de todas las películas de ese año. 

        for year in range (year_inicial, year_inicial+periodo):
            try:
                print(f"Explorando películas del año {year}")
                lista_pelis_anual = todas_las_peliculas_del_anyo(year)
                lista_periodo.extend(lista_pelis_anual)
            except:
                print(f"No ha sido posible obtener los datos. Recomendamos introducir periodos máximos de 5 años")
                return

        encabezados = ["Tipo", "Título", "Año", "Género", "Id_API", "IdOwner"]
        df = pd.DataFrame(lista_periodo, columns = encabezados) 
        escribir_fichero (df, year_inicial, periodo) # Escribimos los datos en fichero .csv
        return df
    
    else:
        print("Los parámetros introducidos no son válidos. \nPor favor, revise el año inicial o el periodo")  


Probamos a extraer películas de un periodo de 15 años. Para que el servidor no se sature, lo hacemos en bloques de 5 años.

- 1995- 1999.
- 2000- 2004.
- 2005-2014


In [10]:
df_pelis_1995_1999 = extraer_datos_peliculas_periodo(1995, 5)

Explorando películas del año 1995
Explorando películas del año 1996
Explorando películas del año 1997
Explorando películas del año 1998
Explorando películas del año 1999
Extracción completa. Puedes consultar tu archivo peliculas_1995_1999.csv


In [11]:
df_pelis_1995_1999.head()

,Tipo,Título,Año,Género,Id_API,IdOwner
0,Movie,Nobat e Asheghi,1995,Drama,54935,tt0100267
1,Movie,¡Que vivan los crotos!,1995,Drama,54937,tt0102739
2,Movie,Paraíso Perdido,1995,Drama,54938,tt0105096
3,Movie,Running Wild,1995,Drama,54939,tt0105298
4,Movie,Las cosas del querer 2ª parte,1995,Drama,54941,tt0106616


In [14]:
df_pelis_2005_2010 = extraer_datos_peliculas_periodo(2005, 5)

Explorando películas del año 2005
Explorando películas del año 2006
Explorando películas del año 2007
Explorando películas del año 2008
Explorando películas del año 2009
Extracción completa. Puedes consultar tu archivo peliculas_2005_2009.csv
